In [1]:
!pip install selenium
!pip install fake_useragent
!pip install requests-html
!pip install webdriver-manager
!pip install --upgrade webdriver-manager

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


In [2]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from multiprocessing import Pool
import random
import json
import os
import pandas as pd
from datetime import datetime

# 2.-Configuración de opciones de Chrome y Creación del objeto WebDriver:

In [3]:
# Crear un objeto UserAgent
ua = UserAgent()

# Crear un objeto Options
chrome_options = Options()
chrome_options.add_argument('--headless')  # Ejecución sin cabeza
chrome_options.add_argument("--incognito")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-setuid-sandbox")
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument(f'user-agent={ua.random}')
chrome_options.headless = True

#
# Crear la instancia del navegador con las opciones definidas
driver = webdriver.Chrome(options=chrome_options)

# Establecer timeout
driver.set_page_load_timeout(20)

# O establecer timeout implicito para encontrar elementos
driver.implicitly_wait(20)

In [4]:

# Definir la URL: Se define la URL de la página de Airbnb para Madrid que se desea analizar.
airbnb_url_madrid = f'https://www.airbnb.es/s/Madrid--España/homes?'

# Conexión: Se utiliza Selenium para abrir la URL en un navegador Chrome controlado por el script.
driver.get(airbnb_url_madrid)
time.sleep(10)  # Esperar a que se carguen los datos

# Obtener el HTML de la página
html = driver.page_source

# Parsear el HTML: Se utiliza BeautifulSoup para parsear el contenido de la respuesta HTTP (que es el HTML de la página web) y crear un objeto BeautifulSoup para facilitar la extracción de datos.
soup = BeautifulSoup(html, 'lxml')


In [5]:
# Imprimir información sobre la respuesta HTTP
print(driver.current_url)  # URL final después de las redirecciones
print(driver.title)        # Título de la página


https://www.airbnb.es/s/Madrid--Espa%C3%B1a/homes
Airbnb | Alquileres vacacionales y alojamientos en Madrid - Comunidad de Madrid


In [6]:
# Extracción de anuncios página 4

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Configuración del WebDriver
chrome_options = Options()
chrome_options.headless = True
# driver = webdriver.Chrome(options=chrome_options)

# URL inicial
airbnb_url_madrid = 'https://www.airbnb.es/s/Madrid--España/homes'
driver.get(airbnb_url_madrid)
time.sleep(5)

# Navegación a la página 4
pagina_deseada = 4
pagina_actual = 1

while pagina_actual < pagina_deseada:
    try:
        boton_siguiente = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@aria-label, 'Siguiente')]"))
        )
        boton_siguiente.click()
        time.sleep(5)
        pagina_actual += 1
    except Exception as e:
        print(f"No se pudo avanzar: {e}")
        break

# Extracción de datos en la página 4
anuncios_pisos_pag4 = []

try:
    elementos_anunciospag4 = driver.find_elements(By.CLASS_NAME, 'c4mnd7m')
    titulos_anuncio = driver.find_elements(By.CLASS_NAME, 't1jojoys')
    subtitulos_anuncio = driver.find_elements(By.CLASS_NAME, 't6mzqp7')
    valoraciones_anuncio = driver.find_elements(By.CSS_SELECTOR, ".r1dxllyb.atm_7l_18pqv07.atm_cp_1ts48j8.dir.dir-ltr")
    precios_noche = driver.find_elements(By.CLASS_NAME, '_1jo4hgw')
    precios_total = driver.find_elements(By.CLASS_NAME, '_tt122m')

    for index, anunciopag4 in enumerate(elementos_anunciospag4):
        href_anuncio = anunciopag4.find_element(By.TAG_NAME, 'a').get_attribute('href')
        titulo = titulos_anuncio[index].text if index < len(titulos_anuncio) else 'N/A'
        subtitulo = subtitulos_anuncio[index].text if index < len(subtitulos_anuncio) else 'N/A'
        valoracion = valoraciones_anuncio[index].text if index < len(valoraciones_anuncio) else 'N/A'
        precio_noche = precios_noche[index].text if index < len(precios_noche) else 'N/A'
        precio_total = precios_total[index].text if index < len(precios_total) else 'N/A'

        anuncios_pisos_pag4.append({
            'titulo': titulo,
            'subtitulo': subtitulo,
            'valoracion': valoracion,
            'precio_noche': precio_noche,
            'precio_total': precio_total,
            'href': href_anuncio
        })

    df = pd.DataFrame(anuncios_pisos_pag4)

except Exception as e:
    print(f"Error al extraer anuncios: {e}")



In [7]:
anuncios_pisos_pag4

[{'titulo': 'Apartamento en Centro Madrid',
  'subtitulo': 'Estudio chic en Calle Carretas',
  'valoracion': 'N/A',
  'precio_noche': '137 € \nnoche',
  'precio_total': '685 € en total',
  'href': 'https://www.airbnb.es/rooms/893203231955325552?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-08-03&check_out=2024-08-08&source_impression_id=p3_1715601919_VycoOF8ljXjY6xap&previous_page_section_name=1000&federated_search_id=19cdee8e-8e4e-4437-9133-29bce2df68ab'},
 {'titulo': 'Apartamento en Centro Madrid',
  'subtitulo': 'Apartamento Fantástico en La Latina y Plaza Mayor',
  'valoracion': 'N/A',
  'precio_noche': '161 € \nnoche',
  'precio_total': '802 € en total',
  'href': 'https://www.airbnb.es/rooms/52440161?adults=1&children=0&enable_m3_private_room=true&infants=0&pets=0&search_mode=regular_search&check_in=2024-11-03&check_out=2024-11-08&source_impression_id=p3_1715601919_99M0w6MFdtBgBEB2&previous_page_section_name=1000&federat